In [1]:
import os
import keras
from keras import layers,models,optimizers, applications
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import time
import shutil
import math

# Download dataset
[Download](https://liveeduisegiunl-my.sharepoint.com/:u:/g/personal/m20200597_novaims_unl_pt/EWfJRtW0BRRNoOmYzb6FviQB3_HP_N-rsnPcxm4J7fxEBQ?e=8Mh1xt)

Download file and unzip in the base_dir directory.

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
TrainSamplePercentage = 70
RandomSeed = 42
base_dir = "C:\\Users\\flopes\\Desktop\\CovidChestXRay" # Change to personal folder

# Organize Images

In [ ]:
ImageBank = "ImageBank"

In [ ]:
ImageBankPath = os.path.join(base_dir, ImageBank)

In [ ]:
def create_dir(dir):
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [ ]:
covid_images_path = os.path.join(ImageBankPath, 'COVID')
noncovid_images_path = os.path.join(ImageBankPath, 'NOCOVID')

len_covid = int(len(os.listdir(covid_images_path)) * TrainSamplePercentage / 100)
len_no_covid = int(len(os.listdir(noncovid_images_path)) * TrainSamplePercentage / 100)

print(f"COVID - Train Sample Size : {len_covid}")
print(f"NO COVID - Train Sample Size : {len_no_covid}")

In [ ]:
train_path =  os.path.join(base_dir, 'train')
validation_path = os.path.join(base_dir, 'validation')

In [ ]:
def CreateTrainAndVal(source_COVID, source_NOCOVID,dest_train,dest_val,TrainSamplePercentage, RandomSeed):
    create_dir(dest_train)
    create_dir(dest_val)
    
    # Create class folder in train
    covid_train_path =  os.path.join(dest_train, 'COVID')
    create_dir(covid_train_path)
    no_covid_train_path = os.path.join(dest_train, 'NOCOVID')
    create_dir(no_covid_train_path)
    
    # Create class folder in val
    covid_val_path =  os.path.join(dest_val, 'COVID')
    create_dir(covid_val_path)
    no_covid_val_path = os.path.join(dest_val, 'NOCOVID')
    create_dir(no_covid_val_path)
    
    
    ImageSplit = train_test_split(os.listdir(source_COVID), test_size=(100-TrainSamplePercentage)/100.0, random_state=RandomSeed)
    
    for image in ImageSplit[0]: # train
        src = os.path.join(source_COVID, image)
        dest = os.path.join(covid_train_path, image)
        shutil.copyfile(src, dest)
        
    for image in ImageSplit[1]: # val
        src = os.path.join(source_COVID, image)
        dest = os.path.join(covid_val_path, image)
        shutil.copyfile(src, dest)

        
    ImageSplit = train_test_split(os.listdir(source_NOCOVID), test_size=(100-TrainSamplePercentage)/100.0, random_state=RandomSeed)
    
    for image in ImageSplit[0]: # train
        src = os.path.join(source_NOCOVID, image)
        dest = os.path.join(no_covid_train_path, image)
        shutil.copyfile(src, dest)
        
    for image in ImageSplit[1]: # val
        src = os.path.join(source_NOCOVID, image)
        dest = os.path.join(no_covid_val_path, image)
        shutil.copyfile(src, dest)

In [ ]:
CreateTrainAndVal(covid_images_path, 
                  noncovid_images_path,
                  train_path,
                  validation_path,
                  TrainSamplePercentage, 
                  RandomSeed)

# Run Custom Model

In [ ]:
# Reset Keras Session
def reset_keras(model):
    sess = tf.compat.v1.keras.backend.get_session()
    tf.compat.v1.keras.backend.clear_session()
    sess.close()
    sess = tf.compat.v1.keras.backend.get_session()

    try:
        del model # this is from global space - change this as you need
    except:
        pass

    # use the same config as you used to create the session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
def run_tests(parameters, experiments):
    print("Running : # Run ", parameters['# Run'])
    model = models.Sequential()
    model.add(layers.Conv2D(32,parameters['filter'],activation=parameters['activation']
                            , input_shape=parameters['input_shape'],padding='same'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,parameters['filter'],activation=parameters['activation'],padding='same'))
    model.add(layers.MaxPooling2D((2,2), padding='same'))
    model.add(layers.Conv2D(128,parameters['filter'],activation=parameters['activation'], padding='same'))
    model.add(layers.MaxPooling2D((2,2), padding='same'))
    model.add(layers.Conv2D(128,parameters['filter'],activation=parameters['activation'], padding='same'))
    model.add(layers.MaxPooling2D((2,2), padding='same'))
    
    if parameters['layers_to_add'] > 0:
        for i in range(parameters['layers_to_add']):
            model.add(layers.Conv2D(128,parameters['filter'],activation=parameters['activation'], padding='same'))
            model.add(layers.MaxPooling2D((2,2), padding='same'))
    
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(512, activation=parameters['activation']))
    model.add(layers.Dense(1, activation=parameters['activation_output']))
    
    if parameters['optimizer'] == 'RMSprop':
        optimizer = optimizers.RMSprop(lr=parameters['learning_rate'])
    else:
        optimizer =  optimizers.Adam(learning_rate=parameters['learning_rate'])

    model.compile(loss=parameters['loss_function'], optimizer=optimizer,metrics=['acc','AUC'])
    
    train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
    test_datagen = ImageDataGenerator(rescale = 1./255, rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
    
    train_generator = train_datagen.flow_from_directory(train_path, target_size=parameters['target_size'],
                                                   batch_size=parameters['batch_size'],
                                                   class_mode='binary')
    
    validation_generator = test_datagen.flow_from_directory(validation_path, target_size=parameters['target_size'],
                                                   batch_size=parameters['batch_size'],
                                                   class_mode='binary')
    
    start_time = time.time()

    step_size_train = train_generator.n//train_generator.batch_size
    step_size_val = validation_generator.n//validation_generator.batch_size    
    
    history = model.fit(train_generator,
                      steps_per_epoch = step_size_train,
                      validation_data = validation_generator,
                      epochs = parameters['epochs'],
                      validation_steps = step_size_val
                     )
    
    
    final_time = (time.time() - start_time)
    parameters['model_name'] = "Custom"
    parameters['execution_time'] = final_time / 60
    parameters['accuracy'] = history.history['acc'][-1]
    parameters['auc'] = history.history['auc'][-1]
    parameters['loss'] = history.history['loss'][-1]
    parameters['val_loss'] = history.history['val_loss'][-1]
    parameters['val_acc'] = history.history['val_acc'][-1]
    parameters['val_auc'] = history.history['val_auc'][-1]
    experiments.append(parameters)
    
    del history
    reset_keras(model)

In [ ]:
experiments = []
parameters_list = []

In [ ]:
#init
parameters_list.append({'batch_size': 50,'filter': (3,3),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(150,150,3), 'loss_function':'binary_crossentropy','target_size':(150,150)
              , 'epochs': 20, 'learning_rate': 1e-4, 'comments': 'Inial Parameters', 'optimizer': 'RMSprop', 'layers_to_add': 0})

In [ ]:
#Change the Image Size
parameters_list.append({'batch_size': 50,'filter': (3,3),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256)
              , 'epochs': 20, 'learning_rate': 1e-4, 'comments': 'Change the image size', 'optimizer': 'RMSprop', 'layers_to_add': 0})

In [ ]:
#Changing the Learning Rate
parameters_list.append({'batch_size': 50,'filter': (3,3),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256)
              , 'epochs': 20, 'learning_rate': 0.01, 'optimizer': 'RMSprop', 'layers_to_add': 0,'comments': "Changing the Learning Rate #1"})

parameters_list.append({'batch_size': 50,'filter': (3,3),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'RMSprop', 'layers_to_add': 0,'comments': "Changing the Learning Rate #2"})


In [ ]:
#Changing the Filter Size
parameters_list.append({'batch_size': 50,'filter': (5,5),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'RMSprop', 'layers_to_add': 0,'comments': "Changing the Filter Size #1"})

parameters_list.append( {'batch_size': 50,'filter': (8,8),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'RMSprop', 'layers_to_add': 0,'comments': "Changing the Filter Size #2"})

parameters_list.append( {'batch_size': 50,'filter': (12,12),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'RMSprop', 'layers_to_add': 0,'comments': "Changing the Filter Size #3"})


In [ ]:
#Increase the number of epochs
parameters_list.append({'batch_size': 50,'filter': (3,3),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256)
              , 'epochs': 30, 'learning_rate': 1e-4, 'comments': 'Change the number of epochs #1', 'optimizer': 'RMSprop', 'layers_to_add': 0})




# CHOOSE THIS ONE
parameters_list.append({'batch_size': 50,'filter': (3,3),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256)
              , 'epochs': 40, 'learning_rate': 1e-4, 'comments': 'Change the number of epochs #2', 'optimizer': 'RMSprop', 'layers_to_add': 0})

In [ ]:
#Adam optimizer
parameters_list.append( {'batch_size': 50,'filter': (8,8),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'adam', 'layers_to_add': 0,'comments': "Testing Adam optimizer"})

In [ ]:
#Adding More Layers
parameters_list.append( {'batch_size': 50,'filter': (8,8),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'adam', 'layers_to_add': 2,'comments': "Testing Adam optimizer"})

In [ ]:
#Change the activation function
parameters_list.append( {'batch_size': 50,'filter': (8,8),'activation': 'relu', 'activation_output': 'softmax',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'adam', 'layers_to_add': 2,'comments': "Change the activation function #1"})

#Adam optimizer
parameters_list.append( {'batch_size': 50,'filter': (8,8),'activation': 'relu', 'activation_output': 'softmax',
              'input_shape':(256,256,3), 'loss_function':'binary_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'adam', 'layers_to_add': 0,'comments': "Change the activation function #2"})

In [ ]:
#Categorical cross entropy
parameters_list.append({'batch_size': 50,'filter': (5,5),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'categorical_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'RMSprop', 'layers_to_add': 0,'comments': "Changing the loss function #1"})

parameters_list.append( {'batch_size': 50,'filter': (8,8),'activation': 'relu', 'activation_output': 'sigmoid',
              'input_shape':(256,256,3), 'loss_function':'categorical_crossentropy','target_size':(256,256), 'epochs': 20
              , 'learning_rate': 0.001, 'optimizer': 'adam', 'layers_to_add': 0,'comments': "Changing the loss function #2"})

In [ ]:
i=1
for parameters in parameters_list:
    parameters['# Run'] = i
    i+=1
    run_tests(parameters,experiments)

# Transfer Learning

In [ ]:
def transferLearning(basemodel,imageprocessor,parameters,experiments):
    
    base_model.trainable = False
    inputs = keras.Input(shape=parameters['input_shape'])
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(2,activation=parameters['activation_output'])(x)
    model = keras.Model(inputs, outputs)

    model.compile(optimizer=optimizers.Adam(learning_rate=parameters['learning_rate']),
                  loss=parameters['loss_function'],
                  metrics=['acc','AUC'])
    
    
    train_datagen = ImageDataGenerator(preprocessing_function=imageprocessor)
    val_datagen = ImageDataGenerator(preprocessing_function=imageprocessor)

    train_generator=train_datagen.flow_from_directory(train_path,
                                                     color_mode='rgb',
                                                     batch_size=parameters['batch_size'],
                                                     class_mode='categorical',
                                                     shuffle=True)

    val_generator=val_datagen.flow_from_directory(validation_path, 
                                                     color_mode='rgb',
                                                     batch_size=parameters['batch_size'],
                                                     class_mode='categorical',
                                                     shuffle=True)
    
    step_size_train = train_generator.n//train_generator.batch_size
    step_size_val = val_generator.n//val_generator.batch_size

    start_time = time.time()
    history = model.fit(train_generator,
                          steps_per_epoch = step_size_train,
                          validation_data = val_generator,
#                           epochs = 1,
                          epochs = parameters['epochs'],
                          validation_steps = step_size_val
                         )
    
    final_time = (time.time() - start_time)

    parameters['execution_time'] = final_time / 60
    parameters['accuracy'] = history.history['acc'][-1]
    parameters['auc'] = history.history['auc'][-1]
    parameters['loss'] = history.history['loss'][-1]
    parameters['val_loss'] = history.history['val_loss'][-1]
    parameters['val_acc'] = history.history['val_acc'][-1]
    parameters['val_auc'] = history.history['val_auc'][-1]
    experiments.append(parameters)
    
    del history
    reset_keras(model)

## XCeption

In [ ]:
base_model = applications.Xception(weights='imagenet',
                                    input_shape=(224, 224, 3),
                                    include_top=False)

imageprocessor = applications.xception.preprocess_input

tf_parameters = {
    '# Run' : len(experiments) + 1,
    'model_name': 'XCeption',
    'activation_output': 'softmax',
    'filter': 'NA',
    'layers_to_add': 'NA',
    'activation': 'NA',
    'input_shape':(224, 224, 3),
    'epochs': 30,
    'batch_size': 100,
    'learning_rate': 1e-4,
    'loss_function' : 'categorical_crossentropy'
    
}

transferLearning(base_model,imageprocessor,tf_parameters,experiments)

# InceptionV3

In [ ]:
base_model = applications.InceptionV3(weights='imagenet',
                                    input_shape=(224, 224, 3),
                                    include_top=False)

imageprocessor = applications.inception_v3.preprocess_input

tf_parameters = {
    '# Run' : len(experiments) + 1,
    'model_name': 'InceptionV3',
    'activation_output': 'softmax',
    'filter': 'NA',
    'layers_to_add': 'NA',
    'activation': 'NA',
    'input_shape':(224, 224, 3),
    'epochs': 30,
    'batch_size': 100,
    'learning_rate': 1e-4,
    'loss_function' : 'categorical_crossentropy'
    
}

transferLearning(base_model,imageprocessor,tf_parameters,experiments)

# DenseNet169

In [ ]:
base_model = applications.DenseNet169(weights='imagenet',
                                    input_shape=(224, 224, 3),
                                    include_top=False)

imageprocessor = applications.densenet.preprocess_input

tf_parameters = {
    '# Run' : len(experiments) + 1,
    'model_name': 'DenseNet169',
    'activation_output': 'softmax',
    'filter': 'NA',
    'layers_to_add': 'NA',
    'activation': 'NA',
    'input_shape':(224, 224, 3),
    'epochs': 30,
    'batch_size': 100,
    'learning_rate': 1e-4,
    'loss_function' : 'categorical_crossentropy'
    
}

transferLearning(base_model,imageprocessor,tf_parameters,experiments)

# VGG16

In [ ]:
base_model = applications.VGG16(weights='imagenet',
                                    input_shape=(224, 224, 3),
                                    include_top=False)

imageprocessor = applications.vgg16.preprocess_input

tf_parameters = {
    '# Run' : len(experiments) + 1,
    'model_name': 'VGG16',
    'activation_output': 'softmax',
    'filter': 'NA',
    'layers_to_add': 'NA',
    'activation': 'NA',
    'input_shape':(224, 224, 3),
    'epochs': 30,
    'batch_size': 50,
    'learning_rate': 1e-4,
    'loss_function' : 'categorical_crossentropy'
    
}

transferLearning(base_model,imageprocessor,tf_parameters,experiments)

In [ ]:
base_model = applications.VGG16(weights='imagenet',
                                    input_shape=(224, 224, 3),
                                    include_top=False)

imageprocessor = applications.vgg16.preprocess_input

tf_parameters = {
    '# Run' : len(experiments) + 1,
    'model_name': 'VGG16',
    'activation_output': 'softmax',
    'filter': 'NA',
    'layers_to_add': 'NA',
    'activation': 'NA',
    'input_shape':(224, 224, 3),
    'epochs': 100,
    'batch_size': 50,
    'learning_rate': 1e-4,
    'loss_function' : 'categorical_crossentropy'
    
}

transferLearning(base_model,imageprocessor,tf_parameters,experiments)

# Agregate Results

In [ ]:
df = pd.DataFrame(experiments)
df

In [ ]:
parameters